In [1]:
import numpy as np
import random
from collections import Counter
from datetime import datetime
import keras
import sys
import io
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
%reload_ext tensorboard

Using TensorFlow backend.


In [2]:
with open('data/Mushrooms.txt', 'r') as mushies:
    mushroom_accounts = mushies.read()

with open('data/Cannabis.txt', 'r') as pot:
    cannabis_accounts = pot.read()
    
with open('data/MDMA.txt', 'r') as molly:
    mdma_accounts = molly.read()
    
with open('data/LSD.txt', 'r') as acid:
    lsd_accounts = acid.read()

In [ ]:
tokenize_text(text):
    

In [5]:
#returns a model trained on whatever text it is given. 
def make_model(text):
    chars = sorted(list(set(text)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    seqlen = 40
    step = seqlen
    sentences = []
    for i in range(0, len(text) - seqlen - 1, step):
        sentences.append(text[i: i + seqlen + 1])

    x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)

    for i, sentence in enumerate(sentences):
        for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
            x[i, t, char_indices[char_in]] = 1
            y[i, t, char_indices[char_out]] = 1


    logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

    model = Sequential()
    model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
    model.add(Dense(len(chars), activation='softmax'))


    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(learning_rate=0.1),
        metrics=['categorical_crossentropy', 'accuracy']
    )


    def sample(preds, temperature=1.0):
        """Helper function to sample an index from a probability array."""
        preds = np.asarray(preds).astype('float64')
        preds = np.exp(np.log(preds) / temperature)  # softmax
        preds = preds / np.sum(preds)                #
        probas = np.random.multinomial(1, preds, 1)  # sample index
        return np.argmax(probas)                     #


    def on_epoch_end(epoch, _):
        """Function invoked at end of each epoch. Prints generated text."""
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - seqlen - 1)

        diversity = 0.5
        
        print('----- diversity:', diversity)
        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    model.fit(x, y,
              batch_size=128,
              epochs=10,
              callbacks=[print_callback])
    
    return model

    #%tensorboard --logdir logs/scalars

In [9]:
mushrooms = make_model(mushroom_accounts[:len(mushroom_accounts)//2])

Epoch 1/10
62452/62452 [==============================] - 74s 1ms/step - loss: 2.0112 - categorical_crossentropy: 2.0112 - accuracy: 0.4705

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "e.  I decide I would be more comfortable"
e.  I decide I would be more comfortable of them, and the manding black to said, the time at a to didn't trip. I started she thought the time, and of the told a half to didn't can the and dont coming and not that was the fact the difficult for the experience of the took and and a so stating the trip and made into the first shifting of what at the trip, that I was colds. I felt a choid ground to my mind of the part and and the more of t
Epoch 2/10
62452/62452 [==============================] - 72s 1ms/step - loss: 1.4922 - categorical_crossentropy: 1.4922 - accuracy: 0.5558

----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: "4, 2000Views: 91,879
[ View as PDF (for "
4, 2000Views: 91,87

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log


e started to some be the mean and the deal it was all a smoked and come on the both was the surface of when the perfect the ground them that we had to little to be a mushrooms to secomed to be the perfect me
Epoch 3/10
62452/62452 [==============================] - 70s 1ms/step - loss: 1.4667 - categorical_crossentropy: 1.4667 - accuracy: 0.5630

----- Generating text after Epoch: 2
----- diversity: 0.5
----- Generating with seed: " decided to cook them all up into one bi"
 decided to cook them all up into one bit of my friends and where I started to take the sitter fresh in the full the friends and the physical creations we were the started to not the trip I can further the world a trip when I didn't be the flactions and mushrooms that I can the trees the beautiful stop took me to trip. It was a been the heart that I was being shrooms the straight many experience the straight the appreasure and the trip 
Epoch 4/10
62452/62452 [==============================] - 69s 1ms/step - loss: 1.

In [7]:
mdma = make_model(mdma_accounts)

Epoch 1/10
41149/41149 [==============================] - 48s 1ms/step - loss: 2.7969 - categorical_crossentropy: 2.7969 - accuracy: 0.3510

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "her girl at the way home and started tal"
her girl at the way home and started talking and I hours offer a as trying it was as to contess that the sleep and the friend that I was sense still that I didn't drug smoked my earlding that I was with a from and started and chatted that I have friends some that I have and and lays and and that I have come and I felt have come things the the does of the right and started this and the talking and and conting the called at there all that
Epoch 2/10
41149/41149 [==============================] - 47s 1ms/step - loss: 1.5614 - categorical_crossentropy: 1.5614 - accuracy: 0.5430

----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: " that I wouldn't tell to anybody (except"
 that I wouldn't te

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log


was very front the go to be go to my good by this probably the early so we walked many subses energy and explain is to really became time pill in a going on my friends in the experience to do in the body and was all good minu
Epoch 6/10
41149/41149 [==============================] - 51s 1ms/step - loss: 1.4632 - categorical_crossentropy: 1.4632 - accuracy: 0.5686

----- Generating text after Epoch: 5
----- diversity: 0.5
----- Generating with seed: "e its size.  My phone felt like the God "
e its size.  My phone felt like the God and that the mind of and difficult coming to my and at this best my second in the coming to the feel my sitter the body.  I have a new seemed to the feel my eyes of a time on my eyes were the mind of the body.  I have been back to be begin in my body took the sension and I am in a club a book to the eyes.

It had a time that I could have to do the high been be in the book in the body. I don't all 
Epoch 7/10
41149/41149 [==============================] - 48s 1